In [1]:
##modules
#%matplotlib widget
#%matplotlib inline

%matplotlib qt
import mne
import numpy as np
import matplotlib
# Establecer un backend interactivo, como 'Qt5Agg', 'GTK3Agg', etc.
# Esto depende de los backends disponibles en tu sistema.

import matplotlib.pyplot as plt

matplotlib.use('Qt5Agg')  # Asegúrate de que este backend está instalado.

import pandas as pd 
import os
import sys

from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs

from os.path import join as pathjoin
from time import time

from pathlib import Path

from autoreject import AutoReject

In [2]:
# Variables path
layer_script = "block"

#process=""
disco="g"

subj = "sub-A2004"
subj_copy=subj+"_copia"

#subj = sys.argv[1] ## name of participant list

# Carpeta general
datadir = Path(f"{disco}:\MOUS_204")

#carpetas generales de datos
mri_dir = datadir / f"{subj}"/"anat"
meg_dir = datadir / f"{subj}"/"meg"

# Carpeta de preprocesado
output_preproc = datadir / "output_preproc"

preproc_path = output_preproc / f"preproc_{layer_script}"
preproc_path.mkdir(parents=True, exist_ok=True)
 
# Carpeta de epocas "sucias"
epochs_path = preproc_path / f"epochs_{layer_script}"
epochs_path.mkdir(parents=True, exist_ok=True) 

# Carpeta de ICA
ICA_path = preproc_path / f"ICA_{layer_script}"
ICA_path.mkdir(parents=True, exist_ok=True)

# Épocas limpias
epochs_clean_path = preproc_path / f"epochs_clean_{layer_script}"
epochs_clean_path.mkdir(parents=True, exist_ok=True)

#epocas evoked
evoked_path = Path(preproc_path) / f"evoked_{layer_script}"
evoked_path.mkdir(parents=True, exist_ok=True)
 

# Definir la carpeta de output_source antes de usarla
output_source = Path(r"g:\MOUS_204\output_source")

source_path = output_source / f"source_{layer_script}"
source_path.mkdir(parents=True, exist_ok=True)

#raw_hsp es el raw con fiducials cargados
raw_hsp_path = source_path / f"raw_hsp"
raw_hsp_path.mkdir(parents=True, exist_ok=True)

# Carpeta de forward solution
fwd_path = source_path / f"fwd"
fwd_path.mkdir(parents=True, exist_ok=True)

# Carpeta de inverse solution
inverse_path = source_path / f"inverse"
inverse_path.mkdir(parents=True, exist_ok=True)


#carpeta analysis
output_analysis= datadir / "output_analysis"
analysis_path = output_analysis / f"analysis_{layer_script}"


In [5]:
from scipy.signal import morlet


ImportError: cannot import name 'morlet' from 'scipy.signal' (c:\Users\UCM\anaconda3\Lib\site-packages\scipy\signal\__init__.py)

In [3]:
import scipy
from neurodsp.aperiodic.irasa import compute_irasa

mne.utils.set_config('SUBJECTS_DIR', r'\\wsl$\Ubuntu-20.04\usr\local\freesurfer\subjects', set_env=True)


ImportError: cannot import name 'morlet' from 'scipy.signal' (c:\Users\UCM\anaconda3\Lib\site-packages\scipy\signal\__init__.py)

In [4]:


# epochs_zinnen_path = epochs_clean_path / f"{subj}_epochs_zinnen_{type_script}"
# epochs_woorden_path = epochs_clean_path / f"{subj}_epochs_woorden_{type_script}"

path_zinnen= evoked_path / f"{subj}_evoked_zinnen_{layer_script}-ave.fif"

evokeds_zinnen = mne.read_evokeds(path_zinnen) 

evoked_zinnen=evokeds_zinnen[0]


Reading g:\MOUS_204\output_preproc\preproc_block\evoked_block\sub-A2004_evoked_zinnen_block-ave.fif ...
    Reading extended channel information
    Found the data of interest:
        t =       0.00 ...   42000.00 ms (fix_zinnen)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
No baseline correction applied


In [ ]:
# ##IGNORE THISSS

# # Compute power spectrum omitting nans


# ##power spectrum omitting nans
# def power_spectrum_nans(epochs, fmin, fmax,nfft, missing="conservative" ):
#     #I´m going to use the raw.comput_psd method as it includes a method to omit bads
#     # transform each epoch into a raw
    
#     #pruebas funcion power_spectrum_nans
#     #epochs.plot()
#     #convierto los epochs en un array
#     epochs_data = epochs.get_data()
#     info=epochs.info
#     sfreq=epochs.info['sfreq']
#     raw_epoch_all=[]
#     psds_all=[]
#     epochs_rejected=[]
#     for i in range(0,len(epochs_data)):
#         # check if there are nans in the data
#         if np.isnan(epochs_data[i]).any():
#             if missing=="conservative":
#                 #mark nans as bad 
#                 data = epochs_data[i]  # Sustituir data por epochs_data[6]

#                 # Detectar los puntos de tiempo donde TODOS los canales tienen NaN
#                 nans_per_timepoint = np.isnan(data).all(axis=0)

#                 # Encontrar los índices donde empiezan los agrupamientos de NaNs
#                 nan_start_indices = np.where((~nans_per_timepoint[:-1]) & (nans_per_timepoint[1:]))[0] + 1
#                 nan_start_sec= nan_start_indices/sfreq

#                 # Encontrar los índices donde terminan los agrupamientos de NaNs
#                 nan_end_indices = np.where((nans_per_timepoint[:-1]) & (~nans_per_timepoint[1:]))[0]
#                 nan_end_sec= nan_end_indices/sfreq
#                 duration = nan_end_indices - nan_start_indices

#                 #i do zero padding in the end so the epochs last the same, having nans or not
#                 nan_duration = np.sum(nans_per_timepoint)
#                 n_channels, n_times=epochs_data[i].shape
#                 zero_padding = np.zeros((n_channels, nan_duration))
#                 print(f"zero padding shape: {zero_padding.shape}, len zero padding: {len(zero_padding[1])},len nans: {nan_duration}")
#                 data = np.hstack([data, zero_padding])

#                 # Mostrar los puntos de tiempo donde empiezan y terminan los agrupamientos de NaNs
#                 print(f"Los agrupamientos de NaNs empiezan en los siguientes puntos de tiempo:, {nan_start_indices}, en los segundos: {nan_start_sec}")
#                 print(f"Los agrupamientos de NaNs terminan en los siguientes puntos de tiempo:, {nan_end_indices}, en los segundos: {nan_end_sec}")

#                 raw_epoch= mne.io.RawArray(data,info)
#                 raw_epoch.set_annotations(mne.Annotations(onset=nan_end_indices,duration= duration ,description='bad'))
#                 raw_epoch_all.append(raw_epoch)
#                 psds=raw_epoch.compute_psd(method='welch', fmin=fmin, fmax=fmax, reject_by_annotation=True, n_fft = int(6 * sfreq))
#                 psds_all.append(psds)
            
#             if missing=="drop":
#                 print(f"epoca {i} contiene nans, se va a omitir")
#                 epochs_rejected.append(i)
#                 continue
            
#         else:
#             raw_epoch= mne.io.RawArray(epochs_data[i],info)
#             raw_epoch_all.append(raw_epoch)

#             psds=raw_epoch.compute_psd(method='welch', fmin=fmin, fmax=fmax, reject_by_annotation=True, n_fft = nfft)
#             psds_all.append(psds)
        
#     return psds_all, epochs_rejected


In [5]:
evoked=evoked_zinnen.copy().crop(tmin=0, tmax=38)

evoked.pick("meg")

Condition,fix_zinnen
Data kind,average
Timepoints,22801 samples
Channels,273 channels
Number of averaged epochs,24
Time range (secs),0.0 – 38.0
Baseline (secs),off


In [ ]:
## obtener power spechtum de el objeto evoked
# psds=evoked.compute_psd(method='welch', fmin=fmin, fmax=fmax, reject_by_annotation=True, n_fft = int(6 * sfreq))
# psds_all.append(psds)
            
# psds,freqs= mne.time_frequency.psd_array_welch(array, sfreq, fmin=0, fmax=inf, n_fft=256, n_overlap=0, 
# #                                    n_per_seg=None, n_jobs=None, average='mean', window='hamming', remove_dc=True, *, 
# #                                    output='power', verbose=None)



##please note THAT THIS IS ONLY FOR EVOKED, YOU NEED TO CALCULATE IT THROUGH EPOCHS

##def compute_psd(self, fmin=0, fmax=np.inf, tmin=None, tmax=None, proj=False)
array=evoked.get_data()
sfreq=evoked.info['sfreq']
fmin=0.5
fmax=40
nfft=1024 # to increase the spectral resolution, , although you can go to 4096 if you want to see more details
njobs=10

##compute psd FOR WHOLE SIGNAL
psds,freqs= mne.time_frequency.psd_array_welch(array, sfreq, fmin=fmin, fmax=fmax, n_fft=nfft, n_jobs=njobs)



Effective window size : 1.707 (s)


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   4 out of  10 | elapsed:    2.3s remaining:    3.5s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:    2.3s remaining:    0.9s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    2.3s finished


In [7]:

# 📌 3️⃣ Calcular el PLE con IRASA
ple_values = []  # Lista para almacenar los valores PLE


# Obtener los datos del canal en todas las épocas
#data = epochs.get_data()[:, ch_idx, :]  # (n_epochs, n_samples)

fmin=0.5
fmax=40

for i in range(0,273):
    # Aplicar IRASA para separar el espectro fractal (1/f)
    freqs, psd_fractal, psd_oscil = compute_irasa(array[i], sfreq, f_range=(fmin, fmax))

    # 📌 4️⃣ Ajustar regresión log-log en el espectro fractal
    log_freqs, log_psd_fractal = np.log10(freqs), np.log10(psd_fractal)

    # Ajuste lineal: log(P) = -β log(f) + c
    slope, intercept = np.polyfit(log_freqs, log_psd_fractal, 1)

    # Guardar el exponente PLE (es el negativo de la pendiente)
    ple_values.append(-slope)

# 📌 5️⃣ Promediar el PLE por canal
ple_values = np.array(ple_values)
print(f"Power Law Exponent (PLE) promedio por canal: {ple_values.mean()}")

NameError: name 'compute_irasa' is not defined

In [ ]:
# ## compute power law exponent for each epoch
# def ple_exponent(psds_all, isplot=False):
#     ##prueba con epochs, ahora has de adaptarlo a la función de power_spectrum_nans 
#     # first you need to obtain the frequencies, 
#     freqs= psds_all[0].freqs
#     #for np.polyfit we need to convert the frequencies to array
#     psds_array = [psds.get_data() for psds in psds_all]
#     psds_avg = [np.mean(epoch_psd, axis=0) for epoch_psd in psds_array]
#     psds_grand_avg = np.mean(psds_avg, axis=0)

#     coeffs= np.polyfit(np.log10(freqs), np.log10(psds_grand_avg), deg=1)
#     PLE= -coeffs[0]
#     if isplot==True:
#         #plot of the power spectrum
#         plt.loglog(freqs, psds_grand_avg,  label='PSD Mean')
#         #plot of the coeffs calculated
#         #its 10**y, where y is the linear regression, ax+b, a is slope, x is log10(freqs) and b is the intercept
#         y=coeffs[1] + coeffs[0]*np.log10(freqs)
#         plt.loglog(freqs, 10**y, 'r--', label=f'Fit PLE = {PLE:.2f}')
#         plt.xlabel('Frequency (Hz)')
#         plt.ylabel('Power Spectral Density (dB/Hz)')
#         plt.title('Power Law Exponent Fit using MNE-Python')
#         plt.legend()
#         plt.grid(True)
#         plt.show()

#         plt.show()



#     return coeffs, PLE, psds_grand_avg